In [1]:
import numpy as np
import tensorflow as tf 
import pickle
from tensorflow.keras import layers , activations , models , preprocessing


In [2]:
import zipfile
with zipfile.ZipFile("new.zip","r") as zip_ref:
    zip_ref.extractall("targetdir")

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, preprocessing, utils
import yaml
import os


In [4]:
dir_path = 'C:/Users/dell/Documents/WISE/new/new/data'
files_list = os.listdir(dir_path + os.sep)# list all files in dir

questions = list()
answers = list()

for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')# read each file in dir
    docs = yaml.safe_load(stream) # full data
    conversations = docs['conversations']#send all docs to conversations
    for con in conversations:#each line
        if len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
#This class allows to vectorize a text corpus, by turning each text into either a 
#sequence of integers (each integer being the index of a token in a dictionary)
tokenizer.fit_on_texts( questions + answers )
#The cat sat on the mat." It will create a dictionary , word_index["the"] = 1; word_index["cat"] = 2
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))




VOCAB SIZE : 1134


In [5]:
!pip install gensim
from gensim.models import Word2Vec#learns rel btwn words automatically

vocab=[]
for word in tokenizer.word_index:
    vocab.append(word)
    
x = max(vocab, key=len)
print(x)
print(vocab.index(x))

def tokenize(sentences):
    tokens_list=[]
    vocabulary=[]
    for sentence in sentences:
        sentence = sentence.lower()
        tokens = sentence.split()
        vocabulary+=tokens
        tokens_list.append(tokens)
    return tokens_list,vocabulary


p=tokenize(questions+answers)
model = Word2Vec(p[0])

#encoder input
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
padded_questions = preprocessing.sequence.pad_sequences(tokenized_questions, maxlen=maxlen_questions, padding='post')
encoder_input_data = np.array( padded_questions )
print(encoder_input_data.shape, maxlen_questions)
#decoder input
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(x) for x in tokenized_answers])
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')
decoder_input_data = np.array(padded_answers)
print(decoder_input_data.shape, maxlen_answers)
#decoder output
tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)):
    tokenized_answers[i]=tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')
one_hot_answers = utils.to_categorical(padded_answers, VOCAB_SIZE)
decoder_output_data = np.array(one_hot_answers)
print(decoder_output_data.shape)


thiazolidinediones
852
(312, 54) 54
(312, 55) 55
(312, 55, 1134)


C:\Users\dell\anaconda3\envs\tensor_flow_env\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [6]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 852 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 852 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 852 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 852 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 852)    966168      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 852)    966168      input_2[0][0]                    
_______________________________________________________________________________________

In [7]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=50 ) 
model.save( 'model.h5' )

Epoch 1/50
7/7 [==============================] - 21s 3s/step - loss: 1.3436 - accuracy: 0.1441
Epoch 2/50
7/7 [==============================] - 20s 3s/step - loss: 1.1870 - accuracy: 0.1671
Epoch 3/50
7/7 [==============================] - 20s 3s/step - loss: 1.0870 - accuracy: 0.1156
Epoch 4/50
7/7 [==============================] - 20s 3s/step - loss: 1.0428 - accuracy: 0.1998
Epoch 5/50
7/7 [==============================] - 20s 3s/step - loss: 1.0095 - accuracy: 0.2007
Epoch 6/50
7/7 [==============================] - 20s 3s/step - loss: 0.9860 - accuracy: 0.2028
Epoch 7/50
7/7 [==============================] - 20s 3s/step - loss: 0.9641 - accuracy: 0.2025
Epoch 8/50
7/7 [==============================] - 20s 3s/step - loss: 0.9571 - accuracy: 0.2084
Epoch 9/50
7/7 [==============================] - 20s 3s/step - loss: 0.9142 - accuracy: 0.2141
Epoch 10/50
7/7 [==============================] - 20s 3s/step - loss: 0.8777 - accuracy: 0.2243
Epoch 11/50
7/7 [======================

In [8]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 852 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 852 ,))
    
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [9]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [38]:
enc_model , dec_model = make_inference_models()

def chatbot_response(msg):
  states_values = enc_model.predict( str_to_tokens(msg) )
  empty_target_seq = np.zeros( ( 1 , 1 ) )
  empty_target_seq[0, 0] = tokenizer.word_index['start']
  stop_condition = False
  decoded_translation = ''
  while not stop_condition :
    dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
    sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
    sampled_word = None
    for word , index in tokenizer.word_index.items() :
      if sampled_word_index == index :
        decoded_translation += ' {}'.format( word )
        sampled_word = word
        
      if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
        stop_condition = True
      empty_target_seq = np.zeros( ( 1 , 1 ) )  
      empty_target_seq[ 0 , 0 ] = sampled_word_index
      states_values = [ h , c ]
  return decoded_translation

import tkinter
from tkinter import *
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END,"You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END,"MedBot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
base.title("MedBot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)
#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="9", width="50", font="Arial",)
ChatLog.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )
#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)
#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
base.mainloop()
